In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import pickle


In [3]:
df=pd.read_csv('labeled_data2.csv')

In [4]:
df.columns

Index(['Unnamed: 0', 'src_ip', 'dst_ip', 'src_port', 'dst_port', 'protocol',
       'timestamp', 'flow_duration', 'flow_byts_s', 'flow_pkts_s',
       'fwd_pkts_s', 'bwd_pkts_s', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'pkt_len_max', 'pkt_len_min', 'pkt_len_mean',
       'pkt_len_std', 'pkt_len_var', 'fwd_header_len', 'bwd_header_len',
       'fwd_seg_size_min', 'fwd_act_data_pkts', 'flow_iat_mean',
       'flow_iat_max', 'flow_iat_min', 'flow_iat_std', 'fwd_iat_tot',
       'fwd_iat_max', 'fwd_iat_min', 'fwd_iat_mean', 'fwd_iat_std',
       'bwd_iat_tot', 'bwd_iat_max', 'bwd_iat_min', 'bwd_iat_mean',
       'bwd_iat_std', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags',
       'bwd_urg_flags', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
       'psh_flag_cnt', 'ack_flag_c

In [5]:
df.head()

,Unnamed: 0,src_ip,dst_ip,src_port,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,...,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts,Label
0,0,10.215.173.1,157.240.16.32,48924,443,6,2021-09-15 15:33:24,2525190.0,2094.495860,8.316206,...,0.000000,0.0,126.400000,365.909091,0,10,11,1264,4025,benign
1,1,10.215.173.1,10.215.173.2,16658,53,17,2021-09-15 15:33:24,43626.0,4148.902031,45.844221,...,0.000000,0.0,68.000000,113.000000,0,1,1,68,113,benign
2,2,10.215.173.1,157.240.16.53,37292,5222,6,2021-09-15 15:33:24,2504540.0,735.065122,9.183323,...,71225.071225,0.0,70.454545,88.833333,0,11,12,775,1066,benign
3,3,192.168.101.162,172.217.160.162,39604,443,17,2021-09-15 15:33:24,270572.0,1075.499313,18.479370,...,0.000000,0.0,61.666667,53.000000,0,3,2,185,106,benign
4,4,10.215.173.1,192.168.101.238,44848,53,17,2021-09-15 15:33:24,230191.0,981.793380,8.688437,...,0.000000,0.0,77.000000,149.000000,0,1,1,77,149,benign


In [49]:
pkt_len = df['fwd_pkt_len_max']
pd.unique(pkt_len)

array([ 446.,   68.,  297.,   63.,   77.,  208.,   88.,  175.,   60.,
        300.,   59., 1175., 1179., 1441., 1104., 1397., 1500.,  865.,
         67., 1281.,  229., 1477.,  557.,   66.,  931.,   62.,  573.,
         87., 1209., 1340., 1066., 1221., 1235.,   78., 1351., 1279.,
        565., 1205.,   61., 1359., 1230.,  414., 1231.,   64., 1088.,
        311., 1107.,   74., 1072.,  825.,   81.,   83., 1453., 1113.,
       1251., 1283., 1206.,  578.,  954., 1280., 1058., 1232., 1044.,
       1260.,  440.,  318.,   65., 1094., 1060., 1269., 1360.,  628.,
       1378., 1068., 1054.,   71.,   69.,   70.,   72.,  316.,   82.,
        100.,  407.,   57.,  646.,  644.,  641., 1222., 1320.,  955.,
       1093., 1143., 1087.,  956., 1079., 1160., 1106., 1078., 1259.,
       1131., 1112., 1159., 1136., 1276.,  830., 1437., 1273.,  322.,
       1096.,  319.,  454., 1056.,   73.,  686.,  902.,  604.,  688.,
        303., 1494.,  611.,  680.,   75., 1267.,  894., 1194.,  308.,
       1062.,  936.,

In [7]:
init = df['init_fwd_win_byts']
pd.unique(init)

array([65535,     0,  2048,  2546], dtype=int64)

In [8]:
seg = df['fwd_seg_size_min'].tolist()
pd.unique(seg)

array([20,  8])

In [9]:
iat = df['fwd_iat_tot']
pd.unique(iat)


array([ 582767.,       0.,  862174., ..., 1423555.,  238581.,  421291.])

In [10]:
flow = df['flow_duration']
pd.unique(flow)


array([2525190.,   43626., 2504540., ...,  296628.,  163704.,  570809.])

In [11]:
label = df['Label']
pd.unique(label)

array(['benign', 'attack'], dtype=object)

In [50]:
data = [df["fwd_pkt_len_max"],df["init_fwd_win_byts"],df["fwd_seg_size_min"], df["fwd_iat_tot"],df["flow_duration"], df["Label"]]

In [51]:
headers = ["pkt_len","init","seg","iat","flow","label"]

In [52]:
df1 = pd.concat(data, axis=1, keys=headers)

In [53]:
df1

,pkt_len,init,seg,iat,flow,label
0,446.0,65535,20,582767.0,2525190.0,benign
1,68.0,0,8,0.0,43626.0,benign
2,297.0,65535,20,862174.0,2504540.0,benign
3,63.0,0,8,270572.0,270572.0,benign
4,77.0,0,8,0.0,230191.0,benign
...,...,...,...,...,...,...
54961,70.0,0,8,0.0,134195.0,benign
54962,73.0,0,8,0.0,136412.0,benign
54963,1378.0,0,8,238581.0,296628.0,benign
54964,65.0,0,8,0.0,163704.0,benign


## Model Creation

In [54]:
from pycaret.classification import *
from sklearn.preprocessing import LabelEncoder

In [55]:
# label encoding for Y values
Encoder = LabelEncoder()
le = Encoder.fit(df1["label"])
df1["label"] = le.transform(df1["label"])

In [56]:
le.inverse_transform([0])

array(['attack'], dtype=object)

In [57]:
pd.unique(df1['label'])
# 1 = Benign
# 0 = Attack

array([1, 0])

In [58]:
df1.columns

Index(['pkt_len', 'init', 'seg', 'iat', 'flow', 'label'], dtype='object')

In [59]:
df1

,pkt_len,init,seg,iat,flow,label
0,446.0,65535,20,582767.0,2525190.0,1
1,68.0,0,8,0.0,43626.0,1
2,297.0,65535,20,862174.0,2504540.0,1
3,63.0,0,8,270572.0,270572.0,1
4,77.0,0,8,0.0,230191.0,1
...,...,...,...,...,...,...
54961,70.0,0,8,0.0,134195.0,1
54962,73.0,0,8,0.0,136412.0,1
54963,1378.0,0,8,238581.0,296628.0,1
54964,65.0,0,8,0.0,163704.0,1


In [33]:
# sample 10000 points from majority class to create new dataset
df_maj = df1[df1['label']==1]
df_min = df1[df1['label']==0]
df_maj = df_maj.sample(n=10000)
df_maj = pd.concat([df_maj,df_min], axis=0)


ValueError: Cannot take a larger sample than population when 'replace=False'

In [27]:
len(df_maj)

45790

In [60]:
X = df1.drop('label',axis=1)
y = df1['label']

In [61]:
y

0        1
1        1
2        1
3        1
4        1
        ..
54961    1
54962    1
54963    1
54964    1
54965    1
Name: label, Length: 54966, dtype: int32

In [62]:
X

,pkt_len,init,seg,iat,flow
0,446.0,65535,20,582767.0,2525190.0
1,68.0,0,8,0.0,43626.0
2,297.0,65535,20,862174.0,2504540.0
3,63.0,0,8,270572.0,270572.0
4,77.0,0,8,0.0,230191.0
...,...,...,...,...,...
54961,70.0,0,8,0.0,134195.0
54962,73.0,0,8,0.0,136412.0
54963,1378.0,0,8,238581.0,296628.0
54964,65.0,0,8,0.0,163704.0


In [63]:
X['init'].value_counts()

65535    54682
0          281
2048         2
2546         1
Name: init, dtype: int64

In [64]:
# train and test split from newly created dataset 
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=32, shuffle=True)

In [65]:
y_test.value_counts()

0    10715
1      279
Name: label, dtype: int64

In [66]:
# perform smote for the dataset to solve imbalance
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x_train, y_train = oversample.fit_resample(x_train, y_train)


In [67]:
y_train.value_counts()

1    42848
0    42848
Name: label, dtype: int64

In [68]:
x_train

,pkt_len,init,seg,iat,flow
0,208.000000,65535,20,4.543000e+03,2.062015e+06
1,208.000000,65535,20,5.663000e+03,2.024686e+06
2,175.000000,65535,20,5.886000e+03,2.086372e+06
3,208.000000,65535,20,1.591951e+07,1.742774e+07
4,208.000000,65535,20,8.791000e+03,1.957401e+06
...,...,...,...,...,...
85691,371.722723,65535,20,2.241558e+06,3.969109e+06
85692,317.754254,65535,20,6.974216e+05,2.179286e+06
85693,297.000000,65535,20,1.501675e+06,3.208314e+06
85694,682.679301,65535,20,9.312025e+05,9.313601e+05


In [69]:
# concatenate dataset for passing to pycaret
train_dataset = pd.concat([x_train,y_train],axis=1)
test_dataset = pd.concat([x_test,y_test],axis=1)

In [70]:
# train_dataset.drop(['Unnamed: 0'],axis=1,inplace=True)
# test_dataset.drop(['Unnamed: 0'],axis=1,inplace=True)

In [71]:
train_dataset

,pkt_len,init,seg,iat,flow,label
0,208.000000,65535,20,4.543000e+03,2.062015e+06,0
1,208.000000,65535,20,5.663000e+03,2.024686e+06,0
2,175.000000,65535,20,5.886000e+03,2.086372e+06,0
3,208.000000,65535,20,1.591951e+07,1.742774e+07,0
4,208.000000,65535,20,8.791000e+03,1.957401e+06,0
...,...,...,...,...,...,...
85691,371.722723,65535,20,2.241558e+06,3.969109e+06,1
85692,317.754254,65535,20,6.974216e+05,2.179286e+06,1
85693,297.000000,65535,20,1.501675e+06,3.208314e+06,1
85694,682.679301,65535,20,9.312025e+05,9.313601e+05,1


In [72]:
test_dataset

,pkt_len,init,seg,iat,flow,label
38466,175.0,65535,20,18225.0,1600134.0,0
39624,175.0,65535,20,7717.0,2260500.0,0
15118,208.0,65535,20,4121.0,1913340.0,0
29533,208.0,65535,20,9734.0,2039856.0,0
22011,208.0,65535,20,7345.0,1870652.0,0
...,...,...,...,...,...,...
43759,208.0,65535,20,36769.0,2429140.0,0
28641,208.0,65535,20,35490.0,1216453.0,0
16184,175.0,65535,20,9127.0,1988729.0,0
844,175.0,65535,20,5108.0,1856918.0,0


In [73]:
len(train_dataset['init'].value_counts())

1814

In [74]:
# pycaret init
exp = setup(data = train_dataset, target='label', train_size = 1.0, preprocess = True, test_data = test_dataset, session_id=1, use_gpu=True)

,Description,Value
0,session_id,1
1,Target,label
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(85696, 6)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [38]:
xgb_model = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9924,0.9998,0.9932,0.9916,0.9924,0.9848,0.9848
1,0.9924,0.9998,0.9932,0.9916,0.9924,0.9848,0.9848
2,0.9894,0.9997,0.9888,0.9900,0.9894,0.9788,0.9788
3,0.9914,0.9997,0.9902,0.9925,0.9914,0.9827,0.9827
4,0.9911,0.9997,0.9900,0.9923,0.9911,0.9823,0.9823
5,0.9902,0.9996,0.9900,0.9904,0.9902,0.9804,0.9804
6,0.9918,0.9997,0.9914,0.9923,0.9918,0.9837,0.9837
7,0.9918,0.9998,0.9907,0.9930,0.9918,0.9837,0.9837
8,0.9910,0.9997,0.9925,0.9895,0.9910,0.9820,0.9820
9,0.9912,0.9997,0.9911,0.9914,0.9912,0.9825,0.9825


In [75]:
xgb_model = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9996,1.0000,0.9993,1.0000,0.9996,0.9993,0.9993
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9995,1.0000,0.9993,0.9998,0.9995,0.9991,0.9991
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998
8,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998
9,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998


In [43]:
filename = "xbg_model.pkl"
pickle.dump(xgb_model,open(filename,'wb'))

In [54]:
xgb_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=1, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='gpu_hist', use_label_encoder=True,
              validate_parameters=1, verbosity=0)

In [51]:
tuned_xgb = tune_model(xgb_model, optimize = 'AUC')

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:49:16
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [76]:
ada_model = create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9996,1.0000,0.9993,1.0000,0.9996,0.9993,0.9993
1,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9998,0.9999,0.9998,0.9998,0.9998,0.9995,0.9995
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998
9,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998


In [44]:
filename = "ada_model.pkl"
pickle.dump(ada_model,open(filename,'wb'))

In [54]:
tuned_ada = tune_model(ada_model, optimize = 'AUC')

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:49:16
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [77]:
rf_model = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9996,0.9996,0.9993,1.0000,0.9996,0.9993,0.9993
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9996,1.0000,0.9995,0.9998,0.9996,0.9993,0.9993
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [45]:
filename = "rf_model.pkl"
pickle.dump(rf_model,open(filename,'wb'))

In [43]:
tuned_rf = tune_model(rf_model, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9988,1.0000,0.9975,1.0000,0.9988,0.9975,0.9975
4,0.9994,1.0000,0.9988,1.0000,0.9994,0.9988,0.9988
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9994,1.0000,0.9988,1.0000,0.9994,0.9988,0.9988
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [78]:
lgbm_model=create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9996,0.9996,0.9993,1.0000,0.9996,0.9993,0.9993
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,0.9999,1.0000,1.0000,0.9998,0.9999,0.9998,0.9998
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9996,0.9998,0.9995,0.9998,0.9996,0.9993,0.9993
5,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9999,1.0000,1.0000,0.9998,0.9999,0.9998,0.9998
8,0.9999,1.0000,0.9998,1.0000,0.9999,0.9998,0.9998
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [46]:
filename = "lgbm_model.pkl"
pickle.dump(lgbm_model,open(filename,'wb'))

In [45]:
tuned_lgbm = tune_model(lgbm_model, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9994,1.0000,1.0000,0.9988,0.9994,0.9988,0.9988
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9988,0.9996,0.9975,1.0000,0.9988,0.9975,0.9975
4,0.9994,1.0000,0.9988,1.0000,0.9994,0.9988,0.9988
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9994,0.9999,0.9988,1.0000,0.9994,0.9988,0.9988
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,0.9994,1.0000,1.0000,0.9988,0.9994,0.9988,0.9988


In [42]:
lr_model=create_model('lr') #JUST TO CHECK IF ITS SHOWING GOOD RESULTS FOR ALL

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6653,0.8725,0.3844,0.8775,0.5346,0.3307,0.3998
1,0.6739,0.8765,0.4007,0.8832,0.5513,0.3477,0.4152
2,0.6735,0.8699,0.4072,0.8712,0.5550,0.3470,0.4100
3,0.6643,0.8674,0.3909,0.8625,0.5380,0.3286,0.3925
4,0.6690,0.8719,0.3974,0.8698,0.5456,0.3379,0.4024
5,0.6692,0.8743,0.3897,0.8836,0.5409,0.3384,0.4081
6,0.6610,0.8686,0.3735,0.8786,0.5242,0.3219,0.3934
7,0.6645,0.8675,0.3856,0.8718,0.5347,0.3289,0.3963
8,0.6578,0.8738,0.3676,0.8765,0.5179,0.3157,0.3878
9,0.6616,0.8688,0.3809,0.8685,0.5295,0.3232,0.3906


## Manual training

In [32]:
x_test.drop(['Unnamed: 0'],axis=1,inplace=True)
x_train.drop(['Unnamed: 0'],axis=1,inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [36]:
x_test

,ack,init,seg,iat,flow
38466,0,65535,20,18225.0,1600134.0
39624,0,65535,20,7717.0,2260500.0
15118,0,65535,20,4121.0,1913340.0
29533,0,65535,20,9734.0,2039856.0
22011,0,65535,20,7345.0,1870652.0
...,...,...,...,...,...
43759,0,65535,20,36769.0,2429140.0
28641,0,65535,20,35490.0,1216453.0
16184,0,65535,20,9127.0,1988729.0
844,0,65535,20,5108.0,1856918.0


In [38]:
x_test.to_csv("x_test.csv")

In [37]:
y_test

38466    attack
39624    attack
15118    attack
29533    attack
22011    attack
          ...  
43759    attack
28641    attack
16184    attack
844      attack
28144    benign
Name: label, Length: 10994, dtype: object

In [39]:
y_test.to_csv("y_test.csv")

In [34]:
x_train

,ack,init,seg,iat,flow
0,0,65535,20,4.543000e+03,2.062015e+06
1,0,65535,20,5.663000e+03,2.024686e+06
2,0,65535,20,5.886000e+03,2.086372e+06
3,0,65535,20,1.591951e+07,1.742774e+07
4,0,65535,20,8.791000e+03,1.957401e+06
...,...,...,...,...,...
85691,0,65535,20,1.181844e+06,1.183461e+06
85692,0,65535,20,3.453499e+06,5.922173e+06
85693,0,65535,20,1.551207e+06,1.603044e+06
85694,0,65535,20,3.634921e+05,2.116415e+06


### Training Different Models

In [47]:
xgb = XGBClassifier(scale_pos_weight=1)
xgb_clf = xgb.fit(x_train,y_train)
predictions_xgb = xgb_clf.predict(x_test)



In [48]:
x_train

,ack,init,seg,iat,flow
0,0,65535,20,4.543000e+03,2.062015e+06
1,0,65535,20,5.663000e+03,2.024686e+06
2,0,65535,20,5.886000e+03,2.086372e+06
3,0,65535,20,1.591951e+07,1.742774e+07
4,0,65535,20,8.791000e+03,1.957401e+06
...,...,...,...,...,...
85691,0,65535,20,5.791932e+05,1.921215e+06
85692,0,47016,16,7.084979e+07,7.136828e+07
85693,0,65535,20,3.608052e+05,2.099276e+06
85694,0,65535,20,4.166740e+06,4.167027e+06


In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions_xgb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10715
           1       0.85      0.97      0.91       279

    accuracy                           0.99     10994
   macro avg       0.92      0.99      0.95     10994
weighted avg       1.00      0.99      1.00     10994



In [50]:
print("xgb Score -> ",accuracy_score(predictions_xgb, y_test)*100)
print("xgb roc_auc Score -> ",roc_auc_score(y_test,predictions_xgb))
print("f1 score ->", f1_score(y_test,predictions_xgb))

xgb Score ->  99.49972712388575
xgb roc_auc Score ->  0.9852153464559948
f1 score -> 0.9081803005008348


In [51]:
print(classification_report(y_test,predictions_xgb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10715
           1       0.85      0.97      0.91       279

    accuracy                           0.99     10994
   macro avg       0.92      0.99      0.95     10994
weighted avg       1.00      0.99      1.00     10994



In [52]:
from sklearn.metrics import confusion_matrix

In [53]:
conf_mat_xgb = confusion_matrix(y_test,predictions_xgb)
print(conf_mat_xgb)

[[10667    48]
 [    7   272]]


In [56]:
filename = "xgb_model.pkl"
pickle.dump(xgb_clf,open(filename,'wb'))

# ADA BOOST

In [57]:
from sklearn.ensemble import AdaBoostClassifier

In [58]:
ada = AdaBoostClassifier()
ada_clf = ada.fit(x_train,y_train)
predictions_ada = ada_clf.predict(x_test)

In [59]:
print(classification_report(y_test,predictions_ada))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     10715
           1       0.74      0.99      0.85       279

    accuracy                           0.99     10994
   macro avg       0.87      0.99      0.92     10994
weighted avg       0.99      0.99      0.99     10994



In [60]:
print("ada Score -> ",accuracy_score(predictions_ada, y_test)*100)
print("ada roc_auc Score -> ",roc_auc_score(y_test,predictions_ada))
print("ada f1 score ->", f1_score(y_test,predictions_ada))

ada Score ->  99.08131708204475
ada roc_auc Score ->  0.991796078588787
ada f1 score -> 0.8458015267175574


In [61]:
filename = "ada_model.pkl"
pickle.dump(ada_clf,open(filename,'wb'))

# RF

In [62]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier()
rf_clf = rf.fit(x_train,y_train)
predictions_rf = rf_clf.predict(x_test)

In [63]:
print(classification_report(y_test,predictions_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10715
           1       0.86      0.98      0.92       279

    accuracy                           1.00     10994
   macro avg       0.93      0.99      0.96     10994
weighted avg       1.00      1.00      1.00     10994



In [64]:
print("rf Score -> ",accuracy_score(predictions_rf, y_test)*100)
print("rf roc_auc Score -> ",roc_auc_score(y_test,predictions_rf))
print("rf f1 score ->", f1_score(y_test,predictions_rf))

rf Score ->  99.55430234673457
rf roc_auc Score ->  0.9872407789301502
rf f1 score -> 0.9176470588235294


In [65]:
filename = "rf_model.pkl"
pickle.dump(rf_clf,open(filename,'wb'))

# LGBM

In [66]:
import lightgbm

In [67]:
lgbm= lightgbm.LGBMClassifier()
lgbm_clf = lgbm.fit(x_train,y_train)
predictions_lgbm = lgbm_clf.predict(x_test)

In [68]:
lgbm= lightgbm.LGBMClassifier()
lgbm_clf = lgbm.fit(x_train,y_train)
predictions_lgbm = lgbm_clf.predict(x_test)

In [69]:
print("rf Score -> ",accuracy_score(predictions_lgbm, y_test)*100)
print("rf roc_auc Score -> ",roc_auc_score(y_test,predictions_lgbm))
print("rf f1 score ->", f1_score(y_test,predictions_lgbm))

rf Score ->  99.34509732581408
rf roc_auc Score ->  0.9826766148684472
rf f1 score -> 0.8827361563517915


In [70]:
filename = "lgbm_model.pkl"
pickle.dump(lgbm_clf,open(filename,'wb'))

# Running on Whole Dataset


In [74]:
whole_df = df1
whole_df

,ack,init,seg,iat,flow,label
0,1,5840,20,115799309.0,115799309,1
1,1,5840,20,113973933.0,113973933,1
2,1,5840,20,1.0,112,1
3,1,5840,20,105985004.0,105985004,1
4,1,5840,20,1.0,1,1
...,...,...,...,...,...,...
1582676,1,5840,20,1.0,1,1
1582677,1,5840,20,1.0,1,1
1582678,1,5840,20,1.0,1,1
1582679,1,5840,20,1.0,1,1


In [75]:
whole_y = whole_df['label']
whole_x = whole_df.drop(['label'],axis=1)

In [76]:
whole_x

,ack,init,seg,iat,flow
0,1,5840,20,115799309.0,115799309
1,1,5840,20,113973933.0,113973933
2,1,5840,20,1.0,112
3,1,5840,20,105985004.0,105985004
4,1,5840,20,1.0,1
...,...,...,...,...,...
1582676,1,5840,20,1.0,1
1582677,1,5840,20,1.0,1
1582678,1,5840,20,1.0,1
1582679,1,5840,20,1.0,1


In [77]:
whole_x

,ack,init,seg,iat,flow
0,1,5840,20,115799309.0,115799309
1,1,5840,20,113973933.0,113973933
2,1,5840,20,1.0,112
3,1,5840,20,105985004.0,105985004
4,1,5840,20,1.0,1
...,...,...,...,...,...
1582676,1,5840,20,1.0,1
1582677,1,5840,20,1.0,1
1582678,1,5840,20,1.0,1
1582679,1,5840,20,1.0,1


In [78]:
whole_predict_xgb = xgb_clf.predict(whole_x)

In [89]:
print(classification_report(whole_y,whole_predict_xgb))
conf_mat_xgb = confusion_matrix(whole_y,whole_predict_xgb)
print(conf_mat_xgb)

              precision    recall  f1-score   support

           0       0.31      1.00      0.48       392
           1       1.00      1.00      1.00   1582289

    accuracy                           1.00   1582681
   macro avg       0.66      1.00      0.74   1582681
weighted avg       1.00      1.00      1.00   1582681

[[    392       0]
 [    863 1581426]]


In [91]:
whole_predict_ada = ada_clf.predict(whole_x)
print(classification_report(whole_y,whole_predict_ada))
conf_mat_xgb = confusion_matrix(whole_y,whole_predict_ada)
print(conf_mat_xgb)

              precision    recall  f1-score   support

           0       0.46      1.00      0.63       392
           1       1.00      1.00      1.00   1582289

    accuracy                           1.00   1582681
   macro avg       0.73      1.00      0.81   1582681
weighted avg       1.00      1.00      1.00   1582681

[[    391       1]
 [    466 1581823]]


In [99]:
whole_predict_rf = rf_clf.predict(whole_x)
print(classification_report(whole_y,whole_predict_rf))
conf_mat_xgb = confusion_matrix(whole_y,whole_predict_rf)
print(conf_mat_xgb)

              precision    recall  f1-score   support

           0       0.45      1.00      0.62       392
           1       1.00      1.00      1.00   1582289

    accuracy                           1.00   1582681
   macro avg       0.72      1.00      0.81   1582681
weighted avg       1.00      1.00      1.00   1582681

[[    391       1]
 [    479 1581810]]


In [100]:
whole_predict_lgbm = lgbm_clf.predict(whole_x)
print(classification_report(whole_y,whole_predict_lgbm))
conf_mat_xgb = confusion_matrix(whole_y,whole_predict_lgbm)
print(conf_mat_xgb)

              precision    recall  f1-score   support

           0       0.46      1.00      0.63       392
           1       1.00      1.00      1.00   1582289

    accuracy                           1.00   1582681
   macro avg       0.73      1.00      0.81   1582681
weighted avg       1.00      1.00      1.00   1582681

[[    392       0]
 [    467 1581822]]
